# By Ali Moradzade 9831058

In [1]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2022-06-30


# Similarity Modulation

Here we are going to implement another similarity other that the BM25 which is the default in Elastic. We want you to implement a tf-idf similarity and test it with same queries in phase2 so that you can get a sense of how well your Elastic tf-idf works. Follow the instructions and fill where ever it says # TODO.  <br>
You can contact me in case of any problems via Telegram: @mahvash_sp

In [2]:
from elasticsearch import Elasticsearch, helpers
import json
import warnings

In [3]:
import numpy as np
import pandas as pd

In [4]:
# Filter warnings
warnings.filterwarnings('ignore')

After starting your Elasticsearch on your pc (localhost:9200 is the default) we have to connect to it via the following piece of code


In [5]:
# Here we try to connect to Elastic
es = Elasticsearch("http://localhost:9200")

In [6]:
es.ping()

True

## Create tf-idf Index

### Create Index

In [7]:
# Name of index 
sm_index_name = 'tfidf_index'

In [8]:
# Delete index if one does exist
if es.indices.exists(index=sm_index_name):
    es.indices.delete(index=sm_index_name)

# Create index    
es.indices.create(index=sm_index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'tfidf_index'})

### Add documents

#### Load Processed Data

In [9]:
df = pd.read_pickle('../results/preprocessed-phase02.pickle')

In [10]:
df.head()

,id,title,content,tags,date,url,category,preprocessed
0,0,اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا,\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آس...,"[اعلام زمان, قرعه‌کشی, قرعه‌کشی جام, قرعه‌کشی ...",3/15/2022 5:59:27 PM,https://www.farsnews.ir/news/14001224001005/اع...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'کنفدرا..."
1,1,سجادی :حضور تماشاگران در لیگ برتر فوتبال تابع...,\nبه گزارش خبرگزاری فارس، سید حمید سجادی در حا...,"[سجادی, لیگ, فدراسیون, وزیر ورزش]",3/15/2022 5:30:07 PM,https://www.farsnews.ir/news/14001224000982/سج...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'سید': ..."
2,2,محل برگزاری نشست‌های خبری سرخابی‌ها؛ مجیدی در ...,\nبه گزارش خبرگزاری فارس، نشست خبری پیش از مسا...,"[دربی 94, محل برگزاری, خبری سرخابی‌ها, مجیدی, ...",3/15/2022 5:20:01 PM,https://www.farsnews.ir/news/14001224000971/مح...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'خبری':..."
3,3,ماجدی در نشست با صالحی امیری: امیدوارم در این ...,\nبه گزارش خبرگزاری فارس، سید رضا صالحی امیری...,"[کمیته امداد امام خمینی (ره), کمیته ملی المپیک...",3/15/2022 5:18:00 PM,https://www.farsnews.ir/news/14001224000964/ما...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'سید': ..."
4,4,لیگ‌برتر بسکتبال|‌ نخستین پیروزی شهرداری گرگان...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، در نخس...,"[بسکتبال, لیگ برتر بسکتبال, شهرداری گرگان, تیم...",3/15/2022 5:16:41 PM,https://www.farsnews.ir/news/14001224000947/لی...,sports,"{'گزارش': 1, 'خبرنگار': 1, 'ورزشی': 1, 'خبرگزا..."


#### Making preprocessed column string

In [11]:
def join_spacewise(dictionary):
    result = ''
    for term in dictionary.keys():
        result += ' ' + term
        
    return result

In [12]:
df['preprocessed'] = df['preprocessed'].apply(join_spacewise)

#### Converting dataframe to dictionary

In [13]:
dict_df = df.to_dict('records')

#### Converting dictionary to elastic search format

In [14]:
df.columns

Index(['id', 'title', 'content', 'tags', 'date', 'url', 'category',
       'preprocessed'],
      dtype='object')

In [15]:
def generator(df2):
    for c, row in enumerate(dict_df):
        yield {
            '_index': sm_index_name,
            '_id': row.get('id', None),
            '_source': {
                'title': row.get('title', None),
                'content': row.get('content', None),
                'tags': row.get('tags', None),
                'date': row.get('date', None),
                'url': row.get('url', None),
                'category': row.get('category', None),
                'preprocessed': row.get('preprocessed', None),
            }
        }
    raise StopIteration

#### Uploading data to index

#####  Bulk API

Running below cell takes:
```
    CPU times: user 1.14 s, sys: 8.78 ms, total: 1.15 s
    Wall time: 14 s
```

In [16]:
%%time

try:
    res = helpers.bulk(es, generator(dict_df))
    print('working ...')
except Exception as e:
    pass

CPU times: user 1.15 s, sys: 29.1 ms, total: 1.17 s
Wall time: 13.9 s


In [17]:
# Check index
es.count(index = sm_index_name)

ObjectApiResponse({'count': 11500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

### Configuring a similarity

In order to configure a new similarity function you have to change the similarity from the settings api of the index. This can be done via the function 'put_settings' in python. What we do is to change the 'default' similarity function in Elastic so that it uses the replaced similarity instead. Type of this similarity is set to 'scripted' because tf-idf is not among the pre-defined similarity functions in Elastic anymore. As this similarity is a scripted type the source code of it must be written **by you** and passed to it.<br>
> In order for the changes to be applied, first we close the index and change the settings and then reopen it<br>

Write the tf-idf code in a string and pass it as a value to the "source" key. <br>
You can find the variables needed in your code in [Here](https://www.elastic.co/guide/en/elasticsearch/painless/current/painless-similarity-context.html).

In [18]:
source_code = "double tf = Math.sqrt(doc.freq); double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; double norm = 1/Math.sqrt(doc.length); return query.boost * tf * idf * norm;"                                 

You should run the bellow cell two times, otherwise you get error.

In [20]:
# closing the index
es.indices.close(index=sm_index_name)

# applying the settings
es.indices.put_settings(index=sm_index_name, 
                            settings={
                                "similarity": {
                                      "default": {
                                        "type": "scripted",
                                        "script": {
                                             "source": source_code}
                                      }
                                }
                            }
                       )

# reopening the index
es.indices.open(index=sm_index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

### Query

In this section you have to test your index with same queries you tested phase2. The goal here is to observe how different or simillar your tf-idf Elastic implementation works.

In [21]:
# A function that creates appropriate body for our match content type query
def get_query(text):
    query = {
        "match": {
            "preprocessed": text
        }
    }
    return query

In [22]:
queries = [
    'کنفدراسیون',
    'ورزشگاه آزادی',
    'مرحله‌پلی‌آف‌لیگ 2',
    'آسترازنکا',
    'مرحله‌پلی‌آف‌لیگ فیدر ‌ندا‌شهسواری',
    'میقولی',
    'آسترازنکا میقولی؟'
]

In [23]:
all_res_tfidf = []


for q in queries:
    res_tfidf = es.search(index=sm_index_name, query=get_query(q), explain=True, size=5)
    all_res_tfidf.append(dict(res_tfidf))

In [24]:
for res, q in zip(all_res_tfidf, queries):
    print(q)
    for doc in res['hits']['hits']:
        print(doc['_source']['url'])
    print("----------------------------")

کنفدراسیون
https://www.farsnews.ir/news/14001104000657/بازدید-ناظران-AFC-از-استادیوم-آزادی-عکس
https://www.farsnews.ir/news/14001212000268/میزبان-لیگ-قهرمانان-در-غرب-آسیا-مشخص-شد
https://www.farsnews.ir/news/14001027000611/اعلام-ساعت-دیدارهای-تیم-ملی-مقابل-عراق-و-امارات
https://www.farsnews.ir/news/14001020000675/موافقت-AFC-با-درخواست-پرسپولیس
https://www.farsnews.ir/news/14001112000993/واکنش-AFC-به-پیروزی-تیم-ملی-مقابل-شاگردان-مارویک-اماراتی-ها-با-گل
----------------------------
ورزشگاه آزادی
https://www.farsnews.ir/news/14001104000657/بازدید-ناظران-AFC-از-استادیوم-آزادی-عکس
https://www.farsnews.ir/news/14001106000296/برخلاف-شایعات-مطرح-شده-بازی-ایران-و-عراق-با-حضور-0-هزار-تماشاگر
https://www.farsnews.ir/news/14001107000649/بهترین-بازیکن-دیدار-تیم-ملی-مقابل-عراق-معرفی-شد
https://www.farsnews.ir/news/14001007000217/پخش-زنده-نشست-خبری-فرهاد-مجیدی-پیش-از-دیدار-استقلال-و-فولاد
https://www.farsnews.ir/news/14001107000561/حضور-وزیر-ورزش-در-ورزشگاه-آزادی-برای-تماشای-بازی-ایران-و-عراق
-------

# Deleting our created index

In [25]:
if es.indices.exists(index=sm_index_name):
    es.indices.delete(index=sm_index_name)

---